In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl

In [2]:
dfRows = pl.read_parquet("./saved_bolillas_gn.parquet")

display(dfRows.head(10))

id,combo,b1,b2,b3,b4,b5,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,shared_values,shared_values_count,score_row,int_unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,bool,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,i64
"""0102040507""","[1, 2, … 7]",1,2,4,5,7,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.3333,0.0333,[0.0333],1,3.4978,4
"""0102040508""","[1, 2, … 8]",1,2,4,5,8,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.25,0.0333,[0.0333],1,3.4145,5
"""0102040509""","[1, 2, … 9]",1,2,4,5,9,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.2,0.0333,[0.0333],1,3.3645,6
"""0102040510""","[1, 2, … 10]",1,2,4,5,10,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.1667,0.0333,[0.0333],1,3.3312,7
"""0102040511""","[1, 2, … 11]",1,2,4,5,11,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.1429,0.0333,[0.0333],1,3.3074,8
"""0102040512""","[1, 2, … 12]",1,2,4,5,12,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.125,0.0333,[0.0333],1,3.2895,9
"""0102040513""","[1, 2, … 13]",1,2,4,5,13,false,0,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.1111,0.0333,[0.0333],1,3.2756,10
"""0102040514""","[1, 2, … 14]",1,2,4,5,14,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.1,0.0333,[0.0333],1,3.2645,11
"""0102040515""","[1, 2, … 15]",1,2,4,5,15,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.0909,0.0333,[0.0333],1,3.2554,12


# Get random ticket

In [8]:
dfRows.sample(n=1)  

id,combo,b1,b2,b3,b4,b5,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,shared_values,shared_values_count,score_row,int_unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,bool,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,i64
"""0611152728""","[6, 11, … 28]",6,11,15,27,28,true,16,0.1,0.0385,0.0455,0.0526,0.125,0.1,0.0769,0.0417,0.0417,0.125,[0.125],1,0.7469,-28


# ANALYZE

In [4]:
dfRows.group_by(
  'reason'
).agg(pl.count('reason').alias('count'))

reason,count
i64,u32
0,141183
16,68519


In [5]:
def show_b_stats (col: str):
  df_viz = dfRows.group_by(
    col
  ).agg(pl.count(col).alias('count'))

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)


In [6]:
show_b_stats('score_left_b2')
show_b_stats('score_r8_b2')


score_left_b2,count
f64,u32
0.0476,6993
0.125,158
0.0333,24568
0.0588,3284
0.0769,1130
…,…
0.1667,46
0.0909,580
0.0556,4088


alt.Chart(...)

score_r8_b2,count
f64,u32
0.125,6804
0.0476,10093
0.0769,11893
0.0588,12294
0.0333,634
…,…
0.1667,4278
0.0909,9415
0.0526,12083


alt.Chart(...)

In [7]:
show_b_stats('score_left_b4')
show_b_stats('score_r8_b4')

score_left_b4,count
f64,u32
0.0476,9415
0.125,7142
0.0769,12083
0.0588,12201
0.0333,586
…,…
0.1667,4549
0.0909,10093
0.0526,11893


alt.Chart(...)

score_r8_b4,count
f64,u32
0.0476,7133
0.125,156
0.0588,3277
0.0333,26462
0.0769,1015
…,…
0.0385,15745
0.0909,576
0.0556,4049


alt.Chart(...)